# Serve Mistral 7B Instruct v0.2 using Triton Inference Server with vLLM

This notebook shows how to serve [Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) model using [Triton Inference Server](https://github.com/triton-inference-server) with [vLLM backend](https://github.com/triton-inference-server/vllm_backend/tree/main).

## Setup and Imports

In [ ]:
! pip install kubernetes

In [ ]:
import os
import subprocess
import time
from kubernetes import client, config

# Load Kubernetes configuration
config.load_kube_config()
v1 = client.CoreV1Api()

def find_matching_helm_pods(release_name, namespace='kubeflow-user-example-com'):
    """Find pods managed by a specific Helm release"""
    helm_pods = v1.list_namespaced_pod(
        namespace=namespace
    )

    matching_pods = []
    for pod in helm_pods.items:
        if (pod.metadata.annotations and
            pod.metadata.annotations.get('app.kubernetes.io/managed-by') == 'Helm' and 
            pod.metadata.annotations.get('app.kubernetes.io/instance') == release_name):
            matching_pods.append(pod)

    return matching_pods

def wait_for_helm_release_pods(release_name, namespace='kubeflow-user-example-com', timeout=1800):
    """Wait for all pods in a helm release to complete successfully"""
    print(f"Waiting for pods in release '{release_name}' to complete...")
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        try:
            matching_pods = find_matching_helm_pods(release_name, namespace)
            
            if not matching_pods:
                print(f"No pods found in Hem release: {release_name} waiting...")
                time.sleep(60)
                continue
            
            all_completed = True
            for pod in matching_pods:
                status = pod.status.phase
                print(f"Pod {pod.metadata.name}: {status}")
                
                if status in ['Pending', 'Running']:
                    all_completed = False
                elif status == 'Failed':
                    print(f"Pod {pod.metadata.name} failed!")
                    return False
            
            if all_completed:
                print("All pods completed successfully!")
                return True
                
        except Exception as e:
            print(f"Error checking pods: {e}")
        
        time.sleep(60)
    
    print(f"Timeout waiting for pods to complete")
    return False

# Set working directory
os.chdir(os.path.expanduser('~/amazon-eks-machine-learning-with-terraform-and-kubeflow'))
print(f"Working directory: {os.getcwd()}")

## Step 1: Download Hugging Face Mistral 7B Instruct v0.2 Model Weights

**Note:** Set your Hugging Face token below before running cell.

In [ ]:
# Replace with your actual Hugging Face token
HF_TOKEN = None
assert HF_TOKEN, "Please set HF_TOKEN"

cmd = [
    'helm', 'install', '--debug', 'triton-server-mistral-7b-instruct-v02-vllm',
    'charts/machine-learning/model-prep/hf-snapshot',
    '--set-json', f'env=[{{"name":"HF_MODEL_ID","value":"mistralai/Mistral-7B-Instruct-v0.2"}},{{"name":"HF_TOKEN","value":"{HF_TOKEN}"}}]',
    '-n', 'kubeflow-user-example-com'
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

In [ ]:
# Wait for model download to complete
wait_for_helm_release_pods('triton-server-mistral-7b-instruct-v02-vllm')

In [ ]:
# Uninstall the model download job
cmd = ['helm', 'uninstall', 'triton-server-mistral-7b-instruct-v02-vllm', '-n', 'kubeflow-user-example-com']
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

## Step 2: Launch Triton Server

In [ ]:
cmd = [
    'helm', 'install', '--debug', 'triton-server-mistral-7b-instruct-v02-vllm',
    'charts/machine-learning/serving/triton-inference-server',
    '-f', 'examples/inference/triton-inference-server/vllm_backend/mistral-7b-instruct-v02/triton_server.yaml',
    '-n', 'kubeflow-user-example-com'
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

In [ ]:
# Wait for Triton server to be ready
def wait_for_triton_server(release_name, namespace='kubeflow-user-example-com', timeout=1800):
    """Wait for Triton server pods to be running and ready"""
    print(f"Waiting for Triton server '{release_name}' to be ready...")
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        try:
            matching_pods = find_matching_helm_pods(release_name, namespace)
            
            if not matching_pods:
                print(f"No pods found in Hem release: {release_name} waiting...")
                time.sleep(60)
                continue
            
            all_ready = True
            for pod in matching_pods:
                status = pod.status.phase
                ready = all(condition.status == 'True' for condition in pod.status.conditions if condition.type == 'Ready')
                print(f"Pod {pod.metadata.name}: {status}, Ready: {ready}")
                
                if status != 'Running' or not ready:
                    all_ready = False
                elif status == 'Failed':
                    print(f"Pod {pod.metadata.name} failed!")
                    return False
            
            if all_ready:
                print("Triton Inference Server Pod is Ready!")
                return True
                
        except Exception as e:
            print(f"Error checking pods: {e}")
        
        time.sleep(60)
    
    print(f"Timeout waiting for Triton Inference Server to be Ready")
    return False

wait_for_triton_server('triton-server-mistral-7b-instruct-v02-vllm')

## Step 3: Check Service Status

In [ ]:
def find_matching_helm_services(release_name, namespace='kubeflow-user-example-com'):
    """Find services managed by a specific Helm release"""
    helm_services = v1.list_namespaced_service(
        namespace=namespace
    )

    matching_services = []
    for service in helm_services.items:
        if (service.metadata.annotations and
            service.metadata.annotations.get('app.kubernetes.io/managed-by') == 'Helm' and
            service.metadata.annotations.get('app.kubernetes.io/instance') == release_name):
            matching_services.append(service)

    return matching_services

services = find_matching_helm_services('triton-server-mistral-7b-instruct-v02-vllm')
print(f"Triton Inference Service service: {services}")

## Step 4: Stop Service

Run this cell when you want to stop the Triton Inference Server service.

In [ ]:

cmd = ['helm', 'uninstall', 'triton-server-mistral-7b-instruct-v02-vllm', '-n', 'kubeflow-user-example-com']
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)